In [17]:
import src.gen_synthetic as gen_data
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq
from scipy.signal import detrend
from scipy import stats
import src.onlineSLE as utility
import periodicity_detection as period
import os

In [18]:
full_path = os.path.expanduser("~/source_code/onlineSLE/dataset/")      

ecg_ts = np.loadtxt(os.path.join(full_path,'TiltECG_200_25000.txt'))
ecg_ts = detrend(ecg_ts)
ecg_ts = stats.zscore(ecg_ts)

sliding_window_size = 1000
answer = np.full(len(ecg_ts), 200) 

## Find length

In [19]:
find_length_result = []
for idx in range(len(ecg_ts) - sliding_window_size + 1):
    W = ecg_ts[idx:sliding_window_size + idx]
    SLE_result = period.find_length(W)
    
    find_length_result.append({'idx_win': sliding_window_size + idx,
                                'answer': answer[sliding_window_size + idx - 1],
                                'result': SLE_result})

print(utility.result_aggregation('ecg', 'find_length',find_length_result))

[{'dataset_name': 'ecg', 'algorithms': 'find_length', 'error_bound': 0, 'accuracy_ratio': 0.053}, {'dataset_name': 'ecg', 'algorithms': 'find_length', 'error_bound': 20, 'accuracy_ratio': 0.928}]


In [20]:
autoperiod_result = []
for idx in range(len(ecg_ts) - sliding_window_size + 1):
    W = ecg_ts[idx:sliding_window_size + idx]
    SLE_result = period.autoperiod(W)
    autoperiod_result.append({'idx_win': sliding_window_size + idx,
                                'answer': answer[sliding_window_size + idx - 1],
                                'result': SLE_result})

print(utility.result_aggregation('ecg', 'autoperiod',autoperiod_result))

[{'dataset_name': 'ecg', 'algorithms': 'autoperiod', 'error_bound': 0, 'accuracy_ratio': 0.058}, {'dataset_name': 'ecg', 'algorithms': 'autoperiod', 'error_bound': 20, 'accuracy_ratio': 0.963}]


In [21]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
sazedR = importr('sazedR')          # sazedR in CRAN


sazedR_result = []
for idx in range(len(ecg_ts) - sliding_window_size + 1):
    W = ecg_ts[idx:sliding_window_size + idx]
    W = robjects.FloatVector(W)
    SLE_result = sazedR.sazed(W)
    SLE_result = SLE_result[0]
    sazedR_result.append({'idx_win': sliding_window_size + idx,
                              'answer': answer[sliding_window_size + idx - 1],
                              'result': SLE_result})

print(utility.result_aggregation('ecg', 'SAZED',sazedR_result))

[{'dataset_name': 'ecg', 'algorithms': 'SAZED', 'error_bound': 0, 'accuracy_ratio': 0.0}, {'dataset_name': 'ecg', 'algorithms': 'SAZED', 'error_bound': 20, 'accuracy_ratio': 0.958}]


## OnlineSLE

In [22]:
## offline mode
onlineSLE_result = []
W = ecg_ts[:sliding_window_size]
mathcal_F = fft(W)
N = len(mathcal_F)
k = np.arange(N)
twiddle = np.exp(2j * np.pi * k / N)   # twiddle for SDFT

xfreq, periodogram_den = utility.peridogram(sliding_window_size, mathcal_F)
peak_index = utility.get_period_hints(periodogram_den)
if peak_index > 1:
    frequency = xfreq[peak_index]
    SLE_result = round(1 / frequency)
else:
    SLE_result = 1
onlineSLE_result.append({'idx_win': sliding_window_size,
                'answer': answer[sliding_window_size - 1],
                'result': SLE_result})

## online mode
for idx, x_t in enumerate(ecg_ts[sliding_window_size:]):
    x_oldest = W[0]
    W = W[1:]
    W = np.append(W, x_t)
    mathcal_F = utility.update_sDFT(mathcal_F, x_oldest, x_t,twiddle)
    xfreq, periodogram_den = utility.peridogram(sliding_window_size, mathcal_F)
    peak_index = utility.get_period_hints(periodogram_den)
    if peak_index > 1:
        frequency = xfreq[peak_index]
        SLE_result = round(1 / frequency)
    else:
        SLE_result = 1
    
    onlineSLE_result.append({'idx_win': 1 + sliding_window_size + idx,
                                'answer': answer[sliding_window_size + idx],
                                'result': SLE_result})
    
print(utility.result_aggregation('ecg', 'onlineSLE', onlineSLE_result))

[{'dataset_name': 'ecg', 'algorithms': 'onlineSLE', 'error_bound': 0, 'accuracy_ratio': 0.862}, {'dataset_name': 'ecg', 'algorithms': 'onlineSLE', 'error_bound': 20, 'accuracy_ratio': 0.863}]
